# Classificador de Imagens Simples

Este notebook mostra como classificar imagens usando uma rede neural pré-treinada.

**O que você vai aprender:**
- Como carregar e usar um modelo pré-treinado
- Pré-processamento de imagens
- Fazer previsões em imagens
- Entender as pontuações de confiança

**Caso de uso:** Identificar objetos em imagens (como "gato", "cachorro", "carro", etc.)

---


## Etapa 1: Importar Bibliotecas Necessárias

Vamos importar as ferramentas que precisamos. Não se preocupe se você ainda não entender todas elas!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Etapa 2: Carregar o Modelo Pré-treinado

Vamos usar **MobileNetV2**, uma rede neural já treinada em milhões de imagens.

Isso é chamado de **Aprendizado por Transferência** - usar um modelo que outra pessoa já treinou!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Etapa 3: Funções Auxiliares

Vamos criar funções para carregar e preparar imagens para o nosso modelo.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Etapa 4: Teste em Imagens de Exemplo

Vamos tentar classificar algumas imagens da internet!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Classifique Cada Imagem


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Etapa 5: Experimente suas próprias imagens!

Substitua o URL abaixo por qualquer URL de imagem que você queira classificar.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 O que Acabou de Acontecer?

1. **Carregamos um modelo pré-treinado** - O MobileNetV2 foi treinado com milhões de imagens
2. **Pré-processamos as imagens** - Redimensionamos e formatamos para o modelo
3. **O modelo fez previsões** - Ele gerou probabilidades para 1000 classes de objetos
4. **Decodificamos os resultados** - Convertendo números em rótulos legíveis para humanos

### Entendendo as Pontuações de Confiança

- **90-100%**: Muito confiante (quase certamente correto)
- **70-90%**: Confiante (provavelmente correto)
- **50-70%**: Moderadamente confiante (pode estar correto)
- **Abaixo de 50%**: Pouco confiante (incerto)

### Por que as previsões podem estar erradas?

- **Ângulo ou iluminação incomuns** - O modelo foi treinado com fotos típicas
- **Objetos múltiplos** - O modelo espera um objeto principal
- **Objetos raros** - O modelo conhece apenas 1000 categorias
- **Imagem de baixa qualidade** - Imagens borradas ou pixeladas são mais difíceis


## 🚀 Próximos Passos

1. **Experimente imagens diferentes:**
   - Encontre imagens no [Unsplash](https://unsplash.com)
   - Clique com o botão direito → "Copiar endereço da imagem" para obter a URL

2. **Faça experimentos:**
   - O que acontece com arte abstrata?
   - Ele consegue reconhecer objetos de diferentes ângulos?
   - Como lida com múltiplos objetos?

3. **Aprenda mais:**
   - Explore as [lições de Visão Computacional](../lessons/4-ComputerVision/README.md)
   - Aprenda a treinar seu próprio classificador de imagens
   - Entenda como funcionam as CNNs (Redes Neurais Convolucionais)

---

## 🎉 Parabéns!

Você acabou de construir um classificador de imagens usando uma rede neural de última geração!

Essa mesma técnica é usada em:
- Google Fotos (organização de suas fotos)
- Carros autônomos (reconhecimento de objetos)
- Diagnóstico médico (análise de raios X)
- Controle de qualidade (detecção de defeitos)

Continue explorando e aprendendo! 🚀



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional feita por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
